In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import catboost as cb
import gc

%matplotlib inline

plt.rc('figure',dpi=100);

In [ ]:
avisos = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisos.csv")
postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulaciones.csv")
postulantes = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [ ]:
vistas = pd.read_csv("Data/fiuba_entrenamiento/pocho/vistas.csv")
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas["total_vistas_postulante"]=0
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

In [ ]:
vistas_postulantes = vistas.groupby("idpostulante").agg({"total_vistas_postulante":"count"})
vistas = pd.merge(vistas,vistas_postulantes, on="idpostulante")
vistas.drop("total_vistas_postulante_x", axis=1,inplace=True)
vistas.rename({"total_vistas_postulante_y": "total_vistas"},axis=1,inplace=True)
vistas.head()

In [ ]:
total_vistas_post = vistas.drop_duplicates("idpostulante").drop(["idaviso","cantidad"],axis = 1)

vistas=vistas.groupby(['idaviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas = pd.merge(vistas,total_vistas_post, on= "idpostulante")
vistas.head()

In [ ]:
vistas["cantidad"] = vistas["cantidad"]/vistas["total_vistas"]
vistas = vistas.drop("total_vistas",axis=1)
vistas.head()

In [7]:
postulaciones2= pd.read_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/azar_total.csv")

postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones2=pd.merge(postulaciones2,postulantes,on='idpostulante')

postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones2=pd.merge(postulaciones2,avisos,on='idaviso')

postulaciones2.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,graduados,estudiantes,conocimientos_especificos,buen_ambiente,indica_sueldo,part_time,full_time,ofrece_crecimiento,ofrece_beneficios,obra_social
0,1112430536,EexaOz,0,MASC,38.0,-,-,-,-,-,...,False,False,True,False,False,False,False,False,False,False
1,1112430536,owaWw0Z,0,FEM,20.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,False
2,1112430536,PmGPRd1,0,MASC,19.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,False
3,1112430536,bO4q324,0,MASC,25.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,False
4,1112430536,MVd5dlr,0,MASC,30.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,False


In [ ]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

len(postulaciones)

In [ ]:
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
print(postulaciones["sepostulo"].mean())
postulaciones.sample(20)


In [ ]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
train=0
test=0

In [ ]:
gc.collect()

In [ ]:
categorical=[features.index('sexo'),features.index('Doctorado'),features.index('Master'),features.index('Otro'),features.index('Posgrado'),features.index('Secundario'),features.index('Terciario/Técnico'),features.index('Universitario'),features.index('nombre_zona'),features.index('tipo_de_trabajo'),features.index('nivel_laboral'),features.index('nombre_area')]
categorical

In [ ]:
c = cb.CatBoostClassifier(depth=13, iterations= 150, l2_leaf_reg= 2, learning_rate= 0.01, use_best_model=True,one_hot_max_size=12,thread_count=12)
#probar con otras L2 regression

c.fit(x_train,y_train,plot=True,eval_set=[(x_test,y_test)],verbose=True,cat_features=categorical)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

In [ ]:
score=c.score(x_test,y_test)*100
print(score)

In [ ]:
joblib.dump(c, "Data/fiuba_entrenamiento/pocho/modelos/Catboost v1.pkl")

In [ ]:
prediccion = pd.read_csv("Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

In [ ]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

In [40]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0. 1.]
100000
100000


In [41]:
prediccion['sepostulo']=y_final2

In [42]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.988637
1,1,0.992872
2,2,0.990226
3,3,0.991474
4,4,0.982249


In [43]:
len(prediccion)

100000

In [44]:
prediccion.to_csv("Data/fiuba_entrenamiento/pocho/predicciones/prediccion10.csv",index=False)
prediccion.sepostulo.mean()

0.9938231342885373